In [1]:
import sys
import os
import glob
import random
import math
import time
import torch; torch.utils.backcompat.broadcast_warning.enabled = True
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim
import torch.backends.cudnn as cudnn; cudnn.benchmark = True
from scipy.fftpack import fft, rfft, fftfreq, irfft, ifft, rfftfreq
from scipy import signal
import numpy as np
import importlib
import cv2

In [2]:
import torchvision
print(torchvision.__version__)
# print(torch.__version__)

0.16.1


In [3]:
eeg_signals_path = "/media/NAS/EEG2IMAGE/eeg_cvpr_2017/data/eeg_5_95_std.pth"
img_path = '/media/NAS/EEG2IMAGE/eeg_cvpr_2017/image'

In [4]:
class EEGDataset:
    
    # Constructor
    def __init__(self, eeg_signals_path, eeg_data_path):
        # Load EEG signals
        print("Start Load...")
        loaded = torch.load(eeg_signals_path)
        # if opt.subject!=0:
        #     self.data = [loaded['dataset'][i] for i in range(len(loaded['dataset']) ) if loaded['dataset'][i]['subject']==0]
        # else:
        self.data=loaded['dataset']        
        self.labels = loaded["labels"]
        self.images = loaded["images"]
        self.image_path = eeg_data_path
        
        # Compute size
        self.size = len(self.data)

    # Get size
    def __len__(self):
        return self.size

    # Get item
    def __getitem__(self, i):
        # Process EEG
        eeg = self.data[i]["eeg"].float().t()
        eeg = eeg[20:460,:]

        # if opt.model_type == "model10":
        #     eeg = eeg.t()
        #     eeg = eeg.view(1,128,460-20)
        # Get label        
        label = self.data[i]["label"]

        # Get Original Image
        image = self.images[self.data[i]["image"]]

        # Return
        return eeg, image, label

# Splitter class
class Splitter:

    def __init__(self, dataset, split_path, split_num=0, split_name="train"):
        # Set EEG dataset
        self.dataset = dataset
        # Load split
        loaded = torch.load(split_path)
        self.split_idx = loaded["splits"][split_num][split_name]
        # Filter data
        self.split_idx = [i for i in self.split_idx if 450 <= self.dataset.data[i]["eeg"].size(1) <= 600]
        # Compute size
        self.size = len(self.split_idx)

    # Get size
    def __len__(self):
        return self.size

    # Get item
    def __getitem__(self, i):
        # Get sample from dataset
        eeg, image, label = self.dataset[self.split_idx[i]]
        # Return
        return eeg, image, label


# Load dataset
dataset = EEGDataset(eeg_signals_path = eeg_signals_path,  eeg_data_path = img_path)
# Create loaders
loaders = {split: DataLoader(Splitter(dataset, split_path = "/media/NAS/EEG2IMAGE/eeg_cvpr_2017/data/block_splits_by_image_all.pth", 
                                      split_num = 0, 
                                      split_name = split), 1, drop_last = False, shuffle = False) for split in ["train", "val", "test"]}


Start Load...


In [4]:

class EEGPreDataset:

    # Constructo
    def __init__(self, eeg_pre_path, eeg_data_path, transforms=None):
        # Load EEG signals
        print("Start Load...")
        # loaded = torch.load(eeg_signals_path)

        # split_loaded = torch.load(split_path)
        # if opt.subject!=0:
        #     self.data = [loaded['dataset'][i] for i in range(len(loaded['dataset']) ) if loaded['dataset'][i]['subject']==0]
        # # else:
        # self.data=loaded['dataset']        
        # self.labels = loaded["labels"]
        # self.images = loaded["images"]
        self.image_path = eeg_data_path
        self.data = glob.glob(os.path.join(eeg_pre_path, "*"))

        # Compute size
        self.dataset_size = len(self.data)

        
        # Transforms
        self.transforms = transforms
        # self.to_tensor  = ToTensor()

    # Get size
    def __len__(self):
        return self.dataset_size

    # Get item
    def __getitem__(self, i):

        loaded = torch.load(self.data[i])
        # Process EEG
        eeg = loaded["eeg"]

        # Get label        
        label = loaded["label"]

        # Get Original Image
        image_name = loaded["image"]
        s, _ = image_name.split("_")
        image = torch.empty((224,224))
        if os.path.exists(os.path.join(self.image_path, s, image_name+".JPEG")):
            image = cv2.imread(os.path.join(self.image_path, s, image_name+".JPEG"))
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) / 255.
        else:
            print(os.path.join(self.image_path, s, image_name+".JPEG"))

        if self.transforms:
            image = self.transforms(image)
        # 
        # image = self.to_tensor(image)
        
        # Return
        return eeg, image, label

dataset = EEGPreDataset(os.path.join(".","preprocessing_data","train") , img_path)
loaders = DataLoader(dataset, 1, drop_last = False, shuffle = False)

Start Load...


In [7]:
import os,sys
from tqdm.notebook  import tqdm

path = os.path.join(".","preprocessing_data")
file_name  = eeg_signals_path.split("/")[-1].replace(".pth", "")

for split in ["train", "val", "test"]:
    for idx, data in tqdm(enumerate(loaders[split]), total = len(loaders[split]), desc = f"{split} data preprocessing..."):
        data = {"eeg":data[0].numpy().squeeze(), "image":data[1][0], "label":data[2].item()}
        torch.save(data, os.path.join(path, split, f"{file_name}_{idx}.pth"))

    

NameError: name 'loaders' is not defined

In [9]:
data['eeg'].shape

(440, 128)

In [ ]:
torch.load(os.path.join(path, split, f"{file_name}_{idx}.pth"))

In [ ]:
image_name = dataset.images[dataset.data[1]["image"]]
s, _ = image_name.split("_")
image = cv2.imread(os.path.join(dataset.image_path, s, image_name+".JPEG"))

In [5]:
for l in loaders:
    pass